In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\recommendation-engine'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    model_name: str
    embedding_dim: int
    epochs: int
    learning_rate: float
    alpha: float
    k: int 

In [5]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.HybridRecommender

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            model_name=config.model_name,
            embedding_dim=params.embedding_dim,
            epochs=params.epochs,
            learning_rate=params.learning_rate,
            alpha=params.alpha,
            k=params.k
        )

In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import joblib
from src.hybrid_recommender import logger

d:\newst\miniconda3\envs\recommendation-engine-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class HybridRecommender:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        self.user_encoder = None
        self.item_encoder = None
        self.organizer_encoder = None
        self.scaler = MinMaxScaler()
        
    def prepare_data(self, df):
        """Prepare data for training with event recommendation data"""
        self.user_encoder = {u: i for i, u in enumerate(df['user_id'].unique())}
        self.item_encoder = {m: i for i, m in enumerate(df['event_id'].unique())}
        self.organizer_encoder = {o: i for i, o in enumerate(df['organizer_id'].unique())}
        
        self.user_decoder = {i: u for u, i in self.user_encoder.items()}
        self.item_decoder = {i: m for m, i in self.item_encoder.items()}
        self.organizer_decoder = {i: o for o, i in self.organizer_encoder.items()}
        
        df['user_id_encoded'] = df['user_id'].map(self.user_encoder)
        df['event_id_encoded'] = df['event_id'].map(self.item_encoder)
        df['organizer_id_encoded'] = df['organizer_id'].map(self.organizer_encoder)
        
        df['interaction_score'] = (
            df['is_booked'] * 0.5 + 
            df['num_bookings'] * 0.1 +
            df['is_liked'] * 0.2 +
            df['num_likes'] * 0.05 +
            df['is_commented'] * 0.3 +
            df['num_comments'] * 0.1
        )
        
        return df

    def build_hybrid_model(self, n_users, n_items, n_organizers):
        """Build hybrid neural network model with organizer information"""
        user_input = Input(shape=(1,), name='user_input')
        user_embedding = Embedding(n_users, self.config.embedding_dim, name='user_embedding')(user_input)
        user_vec = Flatten(name='user_flatten')(user_embedding)
        
        event_input = Input(shape=(1,), name='event_input')
        event_embedding = Embedding(n_items, self.config.embedding_dim, name='event_embedding')(event_input)
        event_vec = Flatten(name='event_flatten')(event_embedding)
        
        organizer_input = Input(shape=(1,), name='organizer_input')
        organizer_embedding = Embedding(n_organizers, self.config.embedding_dim//2, name='organizer_embedding')(organizer_input)
        organizer_vec = Flatten(name='organizer_flatten')(organizer_embedding)
        
        dot_product = Dot(axes=1, name='dot_product')([user_vec, event_vec])
        
        merged = Concatenate()([dot_product, organizer_vec])
        
        dense = Dense(32, activation='relu')(merged)
        dense = Dense(16, activation='relu')(dense)
        
        output = Dense(1, activation='sigmoid', name='output')(dense)
        
        model = Model(inputs=[user_input, event_input, organizer_input], outputs=output)
        model.compile(optimizer=Adam(learning_rate=self.config.learning_rate), 
                     loss='binary_crossentropy',
                     metrics=['accuracy'])
        
        return model

    def train_implicit_model(self, user_items):
        """Train ALS model from implicit"""
        
        weighted = bm25_weight(user_items, K1=100, B=0.8)
        
        model = AlternatingLeastSquares(
            factors=self.config.embedding_dim,
            iterations=self.config.epochs,
            regularization=self.config.alpha,
            random_state=42
        )
        
        model.fit(weighted)
        return model

    def train(self):
        
        """Train hybrid recommendation system"""
        
        df = pd.read_csv(self.config.train_data_path)
        df = self.prepare_data(df)
        
        user_items = csr_matrix(
            (df['interaction_score'].values,
             (df['user_id_encoded'], df['event_id_encoded'])),
            shape=(len(self.user_encoder), len(self.item_encoder))
        )
        
        logger.info("Training implicit ALS model...")
        implicit_model = self.train_implicit_model(user_items)
        
        logger.info("Training neural network hybrid model...")
        n_users = len(self.user_encoder)
        n_items = len(self.item_encoder)
        n_organizers = len(self.organizer_encoder)
        
        nn_model = self.build_hybrid_model(n_users, n_items, n_organizers)
        
        X_user = df['user_id_encoded'].values
        X_event = df['event_id_encoded'].values
        X_organizer = df['organizer_id_encoded'].values
        y = df['interaction_score'].values
        
        y = self.scaler.fit_transform(y.reshape(-1, 1)).flatten()
        
        nn_model.fit(
            [X_user, X_event, X_organizer], y,
            epochs=self.config.epochs,
            batch_size=64,
            validation_split=0.1,
            verbose=1
        )
        
        # Save models
        logger.info("Saving models...")
        models_dir = self.config.root_dir
        joblib.dump(implicit_model, os.path.join(models_dir, 'implicit_model.joblib'))
        nn_model.save(os.path.join(models_dir, 'nn_model.h5'))
        joblib.dump(self.scaler, os.path.join(models_dir, 'scaler.joblib'))
        joblib.dump({
            'user_encoder': self.user_encoder,
            'item_encoder': self.item_encoder,
            'organizer_encoder': self.organizer_encoder
        }, os.path.join(models_dir, 'encoders.joblib'))
        
        logger.info("Training completed!")


In [9]:
try:
    config = ConfigurationManager()
    recommender_config = config.get_model_trainer_config()
    recommender = HybridRecommender(config=recommender_config)
    recommender.train()
except Exception as e:
    logger.exception("Error in training hybrid recommender")
    raise e

[2025-07-01 15:09:57,586: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-01 15:09:57,586: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-01 15:09:57,586: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-01 15:09:57,586: INFO: common: created directory at: artifacts]
[2025-07-01 15:09:57,586: INFO: common: created directory at: artifacts/model_trainer]
[2025-07-01 15:09:57,903: INFO: 2886064907: Training implicit ALS model...]


d:\newst\miniconda3\envs\recommendation-engine-env\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
d:\newst\miniconda3\envs\recommendation-engine-env\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(
100%|██████████| 20/20 [00:00<00:00, 54.29it/s]

[2025-07-01 15:09:58,404: INFO: 2886064907: Training neural network hybrid model...]


Epoch 1/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 18s 8ms/step - accuracy: 0.3047 - loss: 0.3397 - val_accuracy: 0.2990 - val_loss: 0.3015
Epoch 2/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.3019 - loss: 0.3001 - val_accuracy: 0.2990 - val_loss: 0.3021
Epoch 3/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.3033 - loss: 0.2777 - val_accuracy: 0.2990 - val_loss: 0.3090
Epoch 4/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.3014 - loss: 0.2582 - val_accuracy: 0.2990 - val_loss: 0.3153
Epoch 5/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.3024 - loss: 0.2502 - val_accuracy: 0.2990 - val_loss: 0.3201
Epoch 6/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.3010 - loss: 0.2480 - val_accuracy: 0.2990 - val_loss: 0.3271
Epoch 7/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.3043 - loss: 0.2446 - val_accuracy: 0.2990 - val_loss: 0.3283
Epoch 8/20
2110/2110 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.3013 - loss: